In [1]:
## imports
import pandas as pd
import numpy as np
import re
import requests
import yaml


## repeated printouts
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

/Users/vaanigupta/opt/anaconda3/lib/python3.8/site-packages/pandas/core/computation/expressions.py:20: UserWarning: Pandas requires version '2.7.3' or newer of 'numexpr' (version '2.7.1' currently installed).
  from pandas.core.computation.check import NUMEXPR_INSTALLED


# 1. Example 1: no credentials; no wrapper

Site: National Assessment of Education Progress (NAEP)

Documentation: https://www.nationsreportcard.gov/api_documentation.aspx

Base link: https://www.nationsreportcard.gov/DataService/GetAdhocData.aspx 

## 1.1 Query to pull some data

In [2]:
## using their example query of 2011 writing scores separated by gender
## based on here - https://stackoverflow.com/questions/40836749/pythonic-way-of-writing-a-single-line-long-string
## using the ( ) syntax to formulate a long
## string without linebreaks added
example_naep_query = (
'https://www.nationsreportcard.gov/'
'Dataservice/GetAdhocData.aspx?'
'type=data&subject=writing&grade=8&'
'subscale=WRIRP&variable=GENDER&jurisdiction=NP&stattype=MN:MN&Year=2011')


example_naep_query


'https://www.nationsreportcard.gov/Dataservice/GetAdhocData.aspx?type=data&subject=writing&grade=8&subscale=WRIRP&variable=GENDER&jurisdiction=NP&stattype=MN:MN&Year=2011'

In [3]:
## use requests to call the api
naep_resp = requests.get(example_naep_query)
naep_resp
print(type(naep_resp))

## get the json contents of the response 
## here, we're assuming valid response
naep_resp_j = naep_resp.json()
naep_resp_j

## with result, turn it into a dataframe
naep_resp_d = pd.DataFrame(naep_resp_j['result'])
naep_resp_d

<Response [200]>

<class 'requests.models.Response'>


{'status': 200,
 'result': [{'year': 2011,
   'sample': 'R3',
   'yearSampleLabel': '2011',
   'Cohort': 2,
   'CohortLabel': 'Grade 8',
   'stattype': 'MN:MN',
   'subject': 'WRI',
   'grade': 8,
   'scale': 'WRIRP',
   'jurisdiction': 'NP',
   'jurisLabel': 'National public',
   'variable': 'GENDER',
   'variableLabel': 'Gender',
   'varValue': '1',
   'varValueLabel': 'Male',
   'value': 139.099504632971,
   'isStatDisplayable': 1,
   'errorFlag': 0},
  {'year': 2011,
   'sample': 'R3',
   'yearSampleLabel': '2011',
   'Cohort': 2,
   'CohortLabel': 'Grade 8',
   'stattype': 'MN:MN',
   'subject': 'WRI',
   'grade': 8,
   'scale': 'WRIRP',
   'jurisdiction': 'NP',
   'jurisLabel': 'National public',
   'variable': 'GENDER',
   'variableLabel': 'Gender',
   'varValue': '2',
   'varValueLabel': 'Female',
   'value': 158.567104984955,
   'isStatDisplayable': 1,
   'errorFlag': 0}]}

,year,sample,yearSampleLabel,Cohort,CohortLabel,stattype,subject,grade,scale,jurisdiction,jurisLabel,variable,variableLabel,varValue,varValueLabel,value,isStatDisplayable,errorFlag
0,2011,R3,2011,2,Grade 8,MN:MN,WRI,8,WRIRP,NP,National public,GENDER,Gender,1,Male,139.099505,1,0
1,2011,R3,2011,2,Grade 8,MN:MN,WRI,8,WRIRP,NP,National public,GENDER,Gender,2,Female,158.567105,1,0


## 1.2 What happens if there's an error in our query?

In [4]:
## here's a query that from the documentation we know
## won't work since i modified year to 2025 which doesnt
## exist in the data
wrong_naep_query = (
'https://www.nationsreportcard.gov/'
'Dataservice/GetAdhocData.aspx?'
'type=data&subject=writing&grade=8&'
'subscale=WRIRP&variable=GENDER&jurisdiction=NP&stattype=MN:MN&Year=2025')

wrong_naep_query

'https://www.nationsreportcard.gov/Dataservice/GetAdhocData.aspx?type=data&subject=writing&grade=8&subscale=WRIRP&variable=GENDER&jurisdiction=NP&stattype=MN:MN&Year=2025'

In [5]:
## use requests to call the api
naep_wrong_resp = requests.get(wrong_naep_query)
naep_wrong_resp

<Response [200]>

In [7]:
## in the case of this particular api,
## the call returns some response but
## when we try to extract the json containing
## status or results, we get in an error
# naep_wrong_resp.json() # uncomment to see error

### 1.2.2 More all-purpose way of allowing remainder of calls to run: try, except

In [8]:
## putting it in a try; except as general error catching
try:
    results = naep_wrong_resp.json()['result']
except Exception as e:
    print('Failed to get result from API due to error:')
    print(e) # or just: pass

Failed to get result from API due to error:
Invalid control character at: line 1 column 289 (char 288)


### 1.2.3 Can usually also find more targeted way but that varies more across APIs

In [9]:
## if we wanted do more specific error catching,
## see that the status == 400 actually appears here
## so could write if else along those lines
naep_wrong_resp.text
naep_resp.text

if "System.Exception" in naep_wrong_resp.text:
    print("NAEP results not found")

'{"status":400,"result": "System.Exception: The query \'SELECT DISTINCT Framework FROM Cycles WHERE Subject=\'WRI\' AND Cohort=2 AND CONVERT(VARCHAR(10),Year)+Sample IN (\'2025R3\')\' did not return exactly 1 framework. Make sure you can trend the years defined for the given subject and cohort.\r\n   at NRCDataService3.GetAdhocData.GetFramework(NDEContext& ndeContext, String subjectCode, List`1 yearSamples, String cohort) in C:\\projects\\ndecore2015\\NRCDataService2\\GetAdhocData.aspx.cs:line 2091\r\n   at NRCDataService3.GetAdhocData.PopulateBaseOrchestratorRequest() in C:\\projects\\ndecore2015\\NRCDataService2\\GetAdhocData.aspx.cs:line 1781\r\n   at NRCDataService3.GetAdhocData.ConstructRequest_Datapoint() in C:\\projects\\ndecore2015\\NRCDataService2\\GetAdhocData.aspx.cs:line 628\r\n   at NRCDataService3.GetAdhocData.Page_Load(Object sender, EventArgs e) in C:\\projects\\ndecore2015\\NRCDataService2\\GetAdhocData.aspx.cs:line 179"}'

'{"status":200,"result": [{"year":2011,"sample":"R3","yearSampleLabel":"2011","Cohort":2,"CohortLabel":"Grade 8","stattype":"MN:MN","subject":"WRI","grade":8,"scale":"WRIRP","jurisdiction":"NP","jurisLabel":"National public","variable":"GENDER","variableLabel":"Gender","varValue":"1","varValueLabel":"Male","value":139.099504632971,"isStatDisplayable":1,"errorFlag":0},{"year":2011,"sample":"R3","yearSampleLabel":"2011","Cohort":2,"CohortLabel":"Grade 8","stattype":"MN:MN","subject":"WRI","grade":8,"scale":"WRIRP","jurisdiction":"NP","jurisLabel":"National public","variable":"GENDER","variableLabel":"Gender","varValue":"2","varValueLabel":"Female","value":158.567104984955,"isStatDisplayable":1,"errorFlag":0}]}'

NAEP results not found


## Activity 1: writing a function to make multiple, sequential calls

- Say we want to pull the data for grades 4, 8, and 12
- How can we write a function that iterates over a list of those grades and pulls the data for each grade?

**Note**: an ideal function would have arguments for each parameter in the API like subject, subscale, etc. Here we can leave those other parts constant

In [25]:
# your code here
def grades_data(subject, subscale, grade, year):
    try:
        query = ('https://www.nationsreportcard.gov/'
        'Dataservice/GetAdhocData.aspx?'
        'type=data&subject=' + subject +'&grade='
         + str(grade) +
        '&subscale=' + subscale +'&variable=GENDER&jurisdiction=NP&stattype=MN:MN&Year=' + str(year))

        response = requests.get(query)
        return response.json()
    except:
#         print("text", response.text)
        return "An error has occurred-- unable to find any matches"


grades = [4, 8, 12]
subject = 'writing'
subscale = 'WRIRP'
year = 2011

for grade in grades:
    print(grade, grades_data(subject, subscale, grade, year))

4 An error has occurred-- unable to find any matches
8 {'status': 200, 'result': [{'year': 2011, 'sample': 'R3', 'yearSampleLabel': '2011', 'Cohort': 2, 'CohortLabel': 'Grade 8', 'stattype': 'MN:MN', 'subject': 'WRI', 'grade': 8, 'scale': 'WRIRP', 'jurisdiction': 'NP', 'jurisLabel': 'National public', 'variable': 'GENDER', 'variableLabel': 'Gender', 'varValue': '1', 'varValueLabel': 'Male', 'value': 139.099504632971, 'isStatDisplayable': 1, 'errorFlag': 0}, {'year': 2011, 'sample': 'R3', 'yearSampleLabel': '2011', 'Cohort': 2, 'CohortLabel': 'Grade 8', 'stattype': 'MN:MN', 'subject': 'WRI', 'grade': 8, 'scale': 'WRIRP', 'jurisdiction': 'NP', 'jurisLabel': 'National public', 'variable': 'GENDER', 'variableLabel': 'Gender', 'varValue': '2', 'varValueLabel': 'Female', 'value': 158.567104984955, 'isStatDisplayable': 1, 'errorFlag': 0}]}
12 {'status': 200, 'result': [{'year': 2011, 'sample': 'R3', 'yearSampleLabel': '2011', 'Cohort': 3, 'CohortLabel': 'Grade 12', 'stattype': 'MN:MN', 'subje

# 2. Example 2: needs credentials; no wrapper

Create an account here: https://www.yelp.com/developers/v3/manage_app

In [26]:
## get the key
API_KEY = "oSQmQmdSyTgQRttqmgqd8nGBq4OH6bnJSizzWUaqqaHiw8pej3lbrhKSBt0EVQqlxEaNRAbEszD8jvqHMf_P4UhtOXIYy_eSb4ROPOWsWoeBj-y8wjDNOmgLjX8xZXYx"

In [27]:
## use documentation to define what to search
## doc: https://www.yelp.com/developers/documentation/v3/business_search
## write the query 
base_url = "https://api.yelp.com/v3/businesses/search?"
my_name = "restaurants"
my_location = "Hanover,NH,03755"
yelp_genquery = ('{base_url}'
                'term={name}'
                '&location={loc}').format(base_url = base_url,
                name = my_name,
                loc = my_location)

## use requests to call the API; here, we're
## passing it our credentials (structure varies
## by API and telling it to only return 10 results
## (max is 50 at once)
header = {'Authorization': f"Bearer {API_KEY}"}
yelp_genresp = requests.get(yelp_genquery, headers = header)
yelp_genresp

## then, look at structure of response
yelp_genjson = yelp_genresp.json()


<Response [200]>

In [28]:
## example business
yelp_genjson['businesses'][0]

## more automatic way of summarizing but things end up in lists
## within columns for things like categories
yelp_gendf = pd.DataFrame(yelp_genjson['businesses'])
yelp_gendf.head()

{'id': '8ybF6YyRldtZmU9jil4xlg',
 'alias': 'mollys-restaurant-and-bar-hanover',
 'name': "Molly's Restaurant & Bar",
 'image_url': 'https://s3-media4.fl.yelpcdn.com/bphoto/TJLrrA6z-SnPgZfrs2GQNQ/o.jpg',
 'is_closed': False,
 'url': 'https://www.yelp.com/biz/mollys-restaurant-and-bar-hanover?adjust_creative=9dYJu74tHGBXrqi1cwvpUA&utm_campaign=yelp_api_v3&utm_medium=api_v3_business_search&utm_source=9dYJu74tHGBXrqi1cwvpUA',
 'review_count': 508,
 'categories': [{'alias': 'tradamerican', 'title': 'American'},
  {'alias': 'burgers', 'title': 'Burgers'},
  {'alias': 'pizza', 'title': 'Pizza'}],
 'rating': 3.9,
 'coordinates': {'latitude': 43.701144, 'longitude': -72.2894249},
 'transactions': ['delivery'],
 'price': '$$',
 'location': {'address1': '43 South Main St',
  'address2': '',
  'address3': '',
  'city': 'Hanover',
  'zip_code': '03755',
  'country': 'US',
  'state': 'NH',
  'display_address': ['43 South Main St', 'Hanover, NH 03755']},
 'phone': '+16036432570',
 'display_phone': '(

,id,alias,name,image_url,is_closed,url,review_count,categories,rating,coordinates,transactions,price,location,phone,display_phone,distance
0,8ybF6YyRldtZmU9jil4xlg,mollys-restaurant-and-bar-hanover,Molly's Restaurant & Bar,https://s3-media4.fl.yelpcdn.com/bphoto/TJLrrA...,False,https://www.yelp.com/biz/mollys-restaurant-and...,508,"[{'alias': 'tradamerican', 'title': 'American'...",3.9,"{'latitude': 43.701144, 'longitude': -72.2894249}",[delivery],$$,"{'address1': '43 South Main St', 'address2': '...",+16036432570,(603) 643-2570,250.830160
1,XVGEEIH5rVB2QzW-qywcJw,base-camp-cafe-hanover,Base Camp Cafe,https://s3-media1.fl.yelpcdn.com/bphoto/p8_YiE...,False,https://www.yelp.com/biz/base-camp-cafe-hanove...,247,"[{'alias': 'himalayan', 'title': 'Himalayan/Ne...",4.4,"{'latitude': 43.700626, 'longitude': -72.2887803}",[delivery],$$,"{'address1': '3 Lebanon St', 'address2': 'Ste ...",+16036432007,(603) 643-2007,196.139758
2,5WW4g_LRwau29KyjZGLyAA,sawtooth-kitchen-hanover,Sawtooth Kitchen,https://s3-media3.fl.yelpcdn.com/bphoto/61MNG4...,False,https://www.yelp.com/biz/sawtooth-kitchen-hano...,25,"[{'alias': 'chickenshop', 'title': 'Chicken Sh...",4.1,"{'latitude': 43.70158, 'longitude': -72.289641}",[],NaN,"{'address1': '33 S Main St', 'address2': '', '...",+16036435134,(603) 643-5134,242.607552
3,KA8yhrd-ClVYMyOefXdVYg,lous-restaurant-and-bakery-hanover,Lou's Restaurant & Bakery,https://s3-media3.fl.yelpcdn.com/bphoto/VAx8H9...,False,https://www.yelp.com/biz/lous-restaurant-and-b...,383,"[{'alias': 'tradamerican', 'title': 'American'...",4.2,"{'latitude': 43.70143, 'longitude': -72.289001}",[delivery],$$,"{'address1': '30 S Main St', 'address2': '', '...",+16036433321,(603) 643-3321,244.006059
4,jAFLKiVXq9z_h4ZPy0nLMA,the-works-café-hanover,The Works Café,https://s3-media2.fl.yelpcdn.com/bphoto/olpsy-...,False,https://www.yelp.com/biz/the-works-caf%C3%A9-h...,11,"[{'alias': 'bagels', 'title': 'Bagels'}, {'ali...",4.5,"{'latitude': 43.701789, 'longitude': -72.289672}",[],NaN,"{'address1': '25 S Main St', 'address2': '', '...",+16032772082,(603) 277-2082,293.867532


In [29]:
## more data-specific way of summarizing
## we're doing a simple approach and just retaining
## cols that have a simple str structure
## if doing for real, would want to extract things
def clean_yelp_json(one_biz):

    ## restrict to str cols
    d_str = {key:value for key, value in one_biz.items()
             if type(value) == str}
    
    df_str = pd.DataFrame(d_str, index = [d_str['id']])
    return(df_str)

yelp_stronly = [clean_yelp_json(one_b) for one_b in yelp_genjson['businesses']]
yelp_stronly_df = pd.concat(yelp_stronly)

yelp_stronly_df.head(7)


,id,alias,name,image_url,url,price,phone,display_phone
8ybF6YyRldtZmU9jil4xlg,8ybF6YyRldtZmU9jil4xlg,mollys-restaurant-and-bar-hanover,Molly's Restaurant & Bar,https://s3-media4.fl.yelpcdn.com/bphoto/TJLrrA...,https://www.yelp.com/biz/mollys-restaurant-and...,$$,+16036432570,(603) 643-2570
XVGEEIH5rVB2QzW-qywcJw,XVGEEIH5rVB2QzW-qywcJw,base-camp-cafe-hanover,Base Camp Cafe,https://s3-media1.fl.yelpcdn.com/bphoto/p8_YiE...,https://www.yelp.com/biz/base-camp-cafe-hanove...,$$,+16036432007,(603) 643-2007
5WW4g_LRwau29KyjZGLyAA,5WW4g_LRwau29KyjZGLyAA,sawtooth-kitchen-hanover,Sawtooth Kitchen,https://s3-media3.fl.yelpcdn.com/bphoto/61MNG4...,https://www.yelp.com/biz/sawtooth-kitchen-hano...,NaN,+16036435134,(603) 643-5134
KA8yhrd-ClVYMyOefXdVYg,KA8yhrd-ClVYMyOefXdVYg,lous-restaurant-and-bakery-hanover,Lou's Restaurant & Bakery,https://s3-media3.fl.yelpcdn.com/bphoto/VAx8H9...,https://www.yelp.com/biz/lous-restaurant-and-b...,$$,+16036433321,(603) 643-3321
jAFLKiVXq9z_h4ZPy0nLMA,jAFLKiVXq9z_h4ZPy0nLMA,the-works-café-hanover,The Works Café,https://s3-media2.fl.yelpcdn.com/bphoto/olpsy-...,https://www.yelp.com/biz/the-works-caf%C3%A9-h...,NaN,+16032772082,(603) 277-2082
34j_2nRCVQBTKafJvncZlg,34j_2nRCVQBTKafJvncZlg,pine-restaurant-hanover-2,PINE Restaurant,https://s3-media3.fl.yelpcdn.com/bphoto/c2SgUR...,https://www.yelp.com/biz/pine-restaurant-hanov...,$$$,+16036468000,(603) 646-8000
vMyN7JL5cJExJORgIobbQg,vMyN7JL5cJExJORgIobbQg,tuk-tuk-thai-cuisine-hanover,Tuk Tuk Thai Cuisine,https://s3-media4.fl.yelpcdn.com/bphoto/--bUG3...,https://www.yelp.com/biz/tuk-tuk-thai-cuisine-...,$$,+16032779192,(603) 277-9192


# Activity 2: pull restaurants in a different location

- Try running a business search query for your hometown or another place by constructing a query similar to `yelp_genquery` but changing the location parameter
- Other endpoints require feeding what's called the business' fusion id into the API. Take an id from `yelp_stronly.id` and use the documentation here to pull the reviews for that business: https://docs.developer.yelp.com/reference/v3_business_reviews
- **Challenge**: generalize the previous step by writing a function that (1) takes a list of business ids as an input, (2) calls the reviews API for each id, (3) returns the results, and (4) rowbinds all results, i.e. turns them into a single, usable DataFrame

In [32]:
# your code here
base_url = "https://api.yelp.com/v3/businesses/search?"
my_name = "restaurants"
my_location = "Houston,TX,77077"
yelp_genquery = ('{base_url}'
                'term={name}'
                '&location={loc}').format(base_url = base_url,
                name = my_name,
                loc = my_location)

header = {'Authorization': f"Bearer {API_KEY}"}
yelp_genresp = requests.get(yelp_genquery, headers = header)
yelp_genresp

## then, look at structure of response
yelp_genjson = yelp_genresp.json()
yelp_genjson['businesses'][0]

## more automatic way of summarizing but things end up in lists
## within columns for things like categories
yelp_gendf = pd.DataFrame(yelp_genjson['businesses'])
yelp_gendf.head()

<Response [200]>

{'id': 'MRTdhLfHe-Uw-qQnQXxgVg',
 'alias': 'resilience-viet-kitchen-and-bar-houston',
 'name': 'Resilience Viet Kitchen & Bar',
 'image_url': 'https://s3-media3.fl.yelpcdn.com/bphoto/4Ba3LmpzZESpgla9-XyT7w/o.jpg',
 'is_closed': False,
 'url': 'https://www.yelp.com/biz/resilience-viet-kitchen-and-bar-houston?adjust_creative=9dYJu74tHGBXrqi1cwvpUA&utm_campaign=yelp_api_v3&utm_medium=api_v3_business_search&utm_source=9dYJu74tHGBXrqi1cwvpUA',
 'review_count': 141,
 'categories': [{'alias': 'cocktailbars', 'title': 'Cocktail Bars'},
  {'alias': 'vietnamese', 'title': 'Vietnamese'},
  {'alias': 'juicebars', 'title': 'Juice Bars & Smoothies'}],
 'rating': 4.6,
 'coordinates': {'latitude': 29.767836, 'longitude': -95.624604},
 'transactions': ['pickup', 'delivery'],
 'price': '$$',
 'location': {'address1': '1140 Eldridge Pkwy',
  'address2': 'Ste 190',
  'address3': '',
  'city': 'Houston',
  'zip_code': '77077',
  'country': 'US',
  'state': 'TX',
  'display_address': ['1140 Eldridge Pkwy', 

,id,alias,name,image_url,is_closed,url,review_count,categories,rating,coordinates,transactions,price,location,phone,display_phone,distance
0,MRTdhLfHe-Uw-qQnQXxgVg,resilience-viet-kitchen-and-bar-houston,Resilience Viet Kitchen & Bar,https://s3-media3.fl.yelpcdn.com/bphoto/4Ba3Lm...,False,https://www.yelp.com/biz/resilience-viet-kitch...,141,"[{'alias': 'cocktailbars', 'title': 'Cocktail ...",4.6,"{'latitude': 29.767836, 'longitude': -95.624604}","[pickup, delivery]",$$,"{'address1': '1140 Eldridge Pkwy', 'address2':...",+12817417919,(281) 741-7919,1996.648173
1,PqyJx0ndtwBT9wLeFYnhoQ,wula-buhuan-houston-2,Wula Buhuan,https://s3-media1.fl.yelpcdn.com/bphoto/U2JzpT...,False,https://www.yelp.com/biz/wula-buhuan-houston-2...,332,"[{'alias': 'szechuan', 'title': 'Szechuan'}]",4.4,"{'latitude': 29.7547685, 'longitude': -95.622898}","[pickup, delivery]",$$,"{'address1': '13346 Briar Forest Dr', 'address...",+18323288757,(832) 328-8757,895.708322
2,zeFASIGTjBwwPNf0ab86Jw,flip-n-patties-energy-corridor-houston,Flip 'n Patties - Energy Corridor,https://s3-media3.fl.yelpcdn.com/bphoto/TWfkd3...,False,https://www.yelp.com/biz/flip-n-patties-energy...,815,"[{'alias': 'filipino', 'title': 'Filipino'}, {...",4.6,"{'latitude': 29.748863, 'longitude': -95.624445}","[pickup, delivery]",$$,"{'address1': '1809 Eldridge Pkwy', 'address2':...",+18324327869,(832) 432-7869,1083.262042
3,84B9Unm_LNS1dAN1xUQxPQ,la-tartine-de-houston-houston-2,La Tartine de Houston,https://s3-media3.fl.yelpcdn.com/bphoto/TSexoK...,False,https://www.yelp.com/biz/la-tartine-de-houston...,112,"[{'alias': 'french', 'title': 'French'}, {'ali...",4.3,"{'latitude': 29.751378, 'longitude': -95.606345}","[pickup, delivery]",$$,"{'address1': '1570 S Dairy Ashford Rd', 'addre...",+17133606771,(713) 360-6771,749.769174
4,Od4pdoTKE7wdUPL4wJoGIQ,pastelitos-cafe-houston,Pastelitos Cafe,https://s3-media1.fl.yelpcdn.com/bphoto/-fxrR7...,False,https://www.yelp.com/biz/pastelitos-cafe-houst...,483,"[{'alias': 'breakfast_brunch', 'title': 'Break...",4.3,"{'latitude': 29.748886, 'longitude': -95.624641}","[pickup, delivery]",$,"{'address1': '1809 Eldridge Pkwy', 'address2':...",+12815317740,(281) 531-7740,1083.262042


In [55]:
base_url = "https://api.yelp.com/v3/businesses/{}/reviews"
header = {'Authorization': f"Bearer {API_KEY}"}
def get_business_reviews(business_ids):
    total_reviews = []
    for bus_id in business_ids:
        total_query = base_url.format(bus_id)
        yelp_genresp = requests.get(total_query, headers = header)
        yelp_genresp
        yelp_genjson = yelp_genresp.json()['reviews']
        yelp_genjson
        for rev_dict in yelp_genjson:
            rev_dict["business_id"] = bus_id
        
        total_reviews += yelp_genjson

    final_df = pd.DataFrame(total_reviews)
    return final_df

get_business_reviews(["MRTdhLfHe-Uw-qQnQXxgVg", "PqyJx0ndtwBT9wLeFYnhoQ", "zeFASIGTjBwwPNf0ab86Jw"])

,id,url,text,rating,time_created,user,business_id
0,xjE4tUgCALo-ddMtd459qw,https://www.yelp.com/biz/resilience-viet-kitch...,I had lunch here today. You can tell the rest...,5,2024-04-26 12:25:08,"{'id': 'zl_-zhvO1SrTywpQ0pDphg', 'profile_url'...",MRTdhLfHe-Uw-qQnQXxgVg
1,EVkGJQPzffcR9XrGMzGufQ,https://www.yelp.com/biz/resilience-viet-kitch...,I've been here several time since going to the...,5,2024-04-15 08:21:49,"{'id': 'qTk0Y0iuOJKcp9hqEkHmoA', 'profile_url'...",MRTdhLfHe-Uw-qQnQXxgVg
2,a_eFMrbKi2wFfFL-l0TDrA,https://www.yelp.com/biz/resilience-viet-kitch...,Solid Vietnamese spot for lunch or dinner near...,4,2024-03-21 14:34:03,"{'id': 'Icf-gy1YWYck0P3zW3f-pg', 'profile_url'...",MRTdhLfHe-Uw-qQnQXxgVg
3,ntZdkmKg-Unju6OsjXgjQw,https://www.yelp.com/biz/wula-buhuan-houston-2...,"Literally a place called ""if its not spicy, I ...",5,2024-02-27 08:52:00,"{'id': '5cQDH_Z3crvraPAad7moww', 'profile_url'...",PqyJx0ndtwBT9wLeFYnhoQ
4,iR-40oMbUoDdw291Lpc49A,https://www.yelp.com/biz/wula-buhuan-houston-2...,I visited here with a small group. Neither t...,4,2024-01-11 19:50:10,"{'id': 'hFJgRL0m1i1XFXVu7YxGTw', 'profile_url'...",PqyJx0ndtwBT9wLeFYnhoQ
5,faMYtbtbDd-d0_19ARAi2g,https://www.yelp.com/biz/wula-buhuan-houston-2...,"The name is actually a Chinese saying, ""no spi...",5,2023-12-28 15:56:54,"{'id': 'NyZ24GimonjxjrVVWpH5dw', 'profile_url'...",PqyJx0ndtwBT9wLeFYnhoQ
6,vYaWze6ulITGix18rOtZHQ,https://www.yelp.com/biz/flip-n-patties-energy...,Excellent burgers that hit every time. Plus th...,5,2024-03-20 12:19:40,"{'id': 'C0H4YDEXldIyVFbc5nGw8g', 'profile_url'...",zeFASIGTjBwwPNf0ab86Jw
7,TN6snN24y_sDAcrKmjVe9g,https://www.yelp.com/biz/flip-n-patties-energy...,Great burgers omg!! \n\nThe best layers on a b...,5,2024-03-03 08:46:44,"{'id': '1GzYyy7fqPuZsKyYeFZeag', 'profile_url'...",zeFASIGTjBwwPNf0ab86Jw
8,FHqihR5-gk8Q7q2g3mApBQ,https://www.yelp.com/biz/flip-n-patties-energy...,"I walked in thinking it was more of fast food,...",4,2024-03-14 16:04:34,"{'id': 'm5Uz2YUa_U8kR7eDei8NWA', 'profile_url'...",zeFASIGTjBwwPNf0ab86Jw
